# QAP Construction

**Module 10** | 10-snarks-starks

*Lagrange interpolation, vanishing polynomial*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **qap construction**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [R1CS Constraints](10b-r1cs-constraints.ipynb).
- Concepts and notation introduced in the previous notebook.

## From R1CS to QAP

In [ ]:
# Interpolate each column of A, B, C into polynomials
# Using Lagrange interpolation at points 1, 2, ..., m
# TODO: convert R1CS matrices to QAP polynomials

## The Vanishing Polynomial

In [ ]:
# Z(x) = (x-1)(x-2)...(x-m) vanishes at all constraint points
# Valid witness => A(x)*B(x) - C(x) = H(x)*Z(x)
# TODO: compute Z(x) and verify the QAP equation

## Why QAP?

In [ ]:
# QAP allows checking all constraints with a single polynomial identity
# Schwartz-Zippel: check at a random point
# TODO: demonstrate the probabilistic check

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **qap construction**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Groth16 Overview](10d-groth16-overview.ipynb)